In [ ]:
# This notebook illustrates handling the July 2024 Demo of the 6mo Hackathon Scenario 2, described at: https://github.com/DARPA-ASKEM/program-milestones/issues/74

SAVED_RESULTS_DIR = "./out"

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman_demo.example.pde import plot_spatial_timeseries
from IPython.display import HTML
import matplotlib.pyplot as plt
import itertools
from funman import FunmanWorkRequest, EncodingSchedule, POS_INFINITY

import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
from funman import FUNMANConfig
import logging
from funman_demo import summarize_results



RESOURCES = "../resources"

EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet","monthly-demo", "2024-07")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR, "SEIRHD_vacc_petrinet.json"
)
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "SEIRHD_vacc_petrinet_request.json"
)


# %load_ext autoreload
# %autoreload 2

In [ ]:
# N = "Total_population"
STATES = ["S_u", "I_u", "E_u", "S_v", "E_v", "I_v", "H", "R", "D"]
COMPARTMENTS = STATES
MAX_TIME=100
STEP_SIZE=4

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))
    funman_request.structure_parameters[0].schedules= []
    # for t in range(0, MAX_TIME+STEP_SIZE, STEP_SIZE):
    funman_request.structure_parameters[0].schedules.append(EncodingSchedule(timepoints=list(range(0, MAX_TIME+STEP_SIZE, STEP_SIZE))))
    
    for p in funman_request.parameters:
        p.label = "any"

    # # funman_request.config.substitute_subformulas = True
    # funman_request.config.use_transition_symbols = True
    # funman_request.config.use_compartmental_constraints=False
    # funman_request.config.save_smtlib="./out"
    funman_request.config.tolerance = 0.1
    #     funman_request.config.verbosity = 10
    # funman_request.config.dreal_log_level = "info"

    to_synthesize = [p.name for p in funman_request.parameters if p.label == "all"]
    
    results_unconstrained_point = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SIDARTHE Hackathon 6mo",
            case_out_dir=SAVED_RESULTS_DIR,
            dump_plot=True,
            print_last_time=True,
            parameters_to_plot=to_synthesize
    )
    summary = summarize_results(COMPARTMENTS, results_unconstrained_point, ylabel="Population", parameters_to_plot=to_synthesize, synthesized_parameters=to_synthesize)
    print(summary)

In [ ]:
df = results_unconstrained_point.dataframe(points=results_unconstrained_point.parameter_space.points()[-1:])
df[["H"]].plot()
# print(df[40:60])
# df
# results_unconstrained_point.parameter_space.boxes()
# results_unconstrained_point.model.to_dot()

In [ ]:
# from funman import FunmanResults
# out_file = REQUEST_PATH = os.path.join(
#     "..", "out", "cbd1bbf2-e053-43a9-a587-d457fb0d577d.json"
# )
# with open(out_file, "r") as f:
#     results = FunmanResults.model_validate(json.load(f))